In [30]:
import urllib
import pandas as pd
import json
import numpy as np
import requests


In [31]:
base_url = "https://api.stackexchange.com/2.2/search"

In [32]:
def fetch_data(page_start, page_end):
    dataframes = []
    current_page = page_start
    base_url = "https://api.stackexchange.com/2.2/search"
    params = {"order": "desc",
        "sort": "activity",
        "tagged": "javascript",
        "site": "stackoverflow",
         "pagesize": 100,
        "key": "aL0X1YNlgSeQ2MfrTAiLRA((",
         "page": current_page}

    result = requests.get(base_url, params=params)
    data = result.json()
    
    wanted_columns = ["tags", "question_id", "title"]
    while data and current_page <= page_end:
        print(data)
        print("current page", current_page)
        if data["has_more"]:
            items = data["items"]
            for item in items:
                if item["is_answered"]:
                    cleaned_dict = { col: item[col] for col in wanted_columns }
                    dataframes.append(pd.DataFrame(cleaned_dict))
            current_page += 1
            params["page"] = current_page
            result = requests.get(base_url, params=params)
            data = result.json()
        else:
            if(len(dataframes) > 0):
                merged = pd.concat(dataframes)
                return merged
            else:
                return 
        
    if(len(dataframes) > 0):
        merged = pd.concat(dataframes)
        return merged
        


In [33]:
page_start = 1301 #replace
page_end = 1869 #replace
# data = fetch_data(page_start, page_end)

In [8]:
# So far, we have the question IDs. Save the progress!
data.to_pickle("./javascript"+str(page_start)+"_"+str(page_end))

In [34]:
# if your file ever times out, save the pickle file and then read it with this command. Replace the value of page_start (when calling the function to fetch questions ) to where you left off
data = pd.read_pickle("./javascript"+str(page_start)+"_"+str(page_end)+"_with_answers")


In [51]:
def fetch_answer_id(question_id):
    print(question_id)
    base_url = "https://api.stackexchange.com/2.2/questions/" + question_id + "/answers/"
    params = {"order": "desc",
        "sort": "votes",
        "tagged": "javascript",
        "site": "stackoverflow",
         "pagesize": 10,
        "key": "aL0X1YNlgSeQ2MfrTAiLRA(("
        }

    result = requests.get(base_url, params=params)
    data = result.json()
    print(data)
    
    items = data["items"]
    item = items[0]["answer_id"]
    print(item)
    return item
        


In [12]:
# initialize the column where we will be storing the answer ids
if "answer_id" not in data:
    data["answer_id"] = 0

In [36]:
# get the IDs of questions without an answer ID, this ensures that when your requests timeout, 
#you can run this line again and only fetch the questions without an answer

missing_question_ids = data[data["answer_id"] == 0]['question_id'].unique()


In [52]:
# uncomment and run this line if you accidentally closed the file and need to open the answers again

# data = pd.read_pickle("./javascript"+str(page_start)+"_"+str(page_end)+"_with_answers")

missing_question_ids = data[data["answer_id"] == 0]['question_id'].unique()

In [53]:
len(missing_question_ids)

4435

In [56]:
for question in missing_question_ids:
    answer_id = fetch_answer_id(str(question))
    data['answer_id'][data['question_id'] == question] = answer_id

49288987
{'error_id': 502, 'error_message': 'too many requests from this IP, more requests available in 3676 seconds', 'error_name': 'throttle_violation'}


KeyError: 'items'

In [55]:
data.to_pickle("./javascript"+str(page_start)+"_"+str(page_end)+"_with_answers")

In [47]:
data = data[data["question_id"] != 64634934]